<img align="Left" alt="Dados_ao_Cubo" src="../shared_content/DADOS-AO-CUBO-Versão-04-1.png" />
<img align="right" alt="MLNet" src="../shared_content/EJ3KsfJWsAEtDaZ.png" />

## Davi Ramos -> Cientista de Dados 👋
(davi.info@gmail.com)

[![Linkedin Badge](https://img.shields.io/badge/-LinkedIn-blue?style=flat-square&logo=Linkedin&logoColor=white&link=https://www.linkedin.com/in/davi-ramos/)](https://www.linkedin.com/in/davi-ramos/)
[![Twitter Badge](https://img.shields.io/badge/-Twitter-1DA1F2?style=flat-square&logo=Twitter&logoColor=white&link=https://twitter.com/Daviinfo/)](https://twitter.com/Daviinfo/)
<a href="https://github.com/DaviRamos"><img src="https://img.shields.io/github/followers/DaviRamos.svg?label=GitHub&style=social" alt="GitHub"></a>

# ML.Net - Regressão Linear

## Este exemplo irá criar um modelo de regressão linear para prever os preços das residências utilizando os seus dados de tamanho e Preço.

In [3]:
// Instalar os Pacotes ML.NET do Nuget
#r "nuget:Microsoft.ML"

In [4]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;

In [5]:
public class HouseData
{
    public float Size { get; set; }
    public float Price { get; set; }
}

public class Prediction
{
    [ColumnName("Score")]
    public float Price { get; set; }
}

In [6]:
MLContext mlContext = new MLContext();

// 1. Criar os dados de Treinamento
HouseData[] houseData = {
    new HouseData() { Size = 1.1F, Price = 1.2F },
    new HouseData() { Size = 1.9F, Price = 2.3F },
    new HouseData() { Size = 2.8F, Price = 3.0F },
    new HouseData() { Size = 3.4F, Price = 3.7F } };
IDataView trainingData = mlContext.Data.LoadFromEnumerable(houseData);

// 2. Preparação dos dados e Treinamento do pipeline do Modelo
var pipeline = mlContext.Transforms.Concatenate("Features", new[] { "Size" })
    .Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "Price", maximumNumberOfIterations: 100));

// 3. Treinar o Modelo
var model = pipeline.Fit(trainingData);

// 4. Fazer a predição
var size = new HouseData() { Size = 2.5F };
var price = mlContext.Model.CreatePredictionEngine<HouseData, Prediction>(model).Predict(size);

Console.WriteLine($"O Preço previsto para este tamanho é: {size.Size*1000} Metros Quadrados = {price.Price*100:C}k");

O Preço previsto para este tamanho é: 2500 Metros Quadrados = R$275,11k


In [7]:
// Avaliação do modelo       
HouseData[] testHouseData =
{
    new HouseData() { Size = 1.1F, Price = 0.98F },
    new HouseData() { Size = 1.9F, Price = 2.1F },
    new HouseData() { Size = 2.8F, Price = 2.9F },
    new HouseData() { Size = 3.4F, Price = 3.6F }
};

var testHouseDataView = mlContext.Data.LoadFromEnumerable(testHouseData);
var testPriceDataView = model.Transform(testHouseDataView);

var metrics = mlContext.Regression.Evaluate(testPriceDataView, labelColumnName: "Price");

Console.WriteLine($"R^2: {metrics.RSquared:0.##}");
Console.WriteLine($"RMS error: {metrics.RootMeanSquaredError:0.##}");

R^2: 0,97
RMS error: 0,17


In [38]:
// Exemplo de um Gráfico do tipo Scatter com o XPlot

var hx = new List<float>();
var hf = new List<float>();

foreach (var item in houseData)
{
   hx.Add(item.Size);
   hf.Add(item.Price);    
}

var treinoSeries = new Graph.Scatter
{
    name = "Dados de Treinamento",
    x = hx, 
    y = hf
};

var tx = new List<float>();
var tf = new List<float>();

foreach (var item in testHouseData)
{
   tx.Add(item.Size);
   tf.Add(item.Price);    
}

var testeSeries = new Graph.Scatter
{
    name = "Dados de Teste",
    x = tx, 
    y = tf
};

var chart = Chart.Plot(new[] {treinoSeries, testeSeries});
chart.WithTitle("Treinamento vs Teste");
display(chart);

In [41]:
//Salvar o modelo treinado em arquivo
private static string modelPath = @"./HouseModel.zip";

mlContext.Model.Save(model, trainingData.Schema, modelPath); 

## Fluxo de trabalho de código

### O diagrama a seguir representa a estrutura de código do aplicativo e o seu processo iterativo para o desenvolvimento do modelo:

![Fluxo](../shared_content/mldotnet-annotated-workflow.png)

<ul>
   <li>Coletar e carregar dados de treinamento em um objeto IDataView</li>
   <li>Especifique um pipeline de operações para extrair recursos e aplicar um algoritmo de aprendizado de máquina</li>
   <li>Treinar um modelo chamando Fit() no pipeline</li>
   <li>Avaliar o modelo e iterar para melhorar</li>
   <li>Salvar o modelo em um formato binário para uso em um aplicativo</li>
   <li>Carregar o modelo de volta para um objeto ITransformer</li>
   <li>Fazer previsões chamando CreatePredictionEngine.Predict()</li>
</ul>

## Nos próximos artigos iremos nos aprofundar um pouco mais nesses conceitos.